# Tutorial on CasADi and Opti
For more information on CasADi, check out https://web.casadi.org/docs/

For IPOPT options, check out https://coin-or.github.io/Ipopt/OPTIONS.html

In [2]:
from casadi import *
import matplotlib.pyplot as plt
from matplotlib.animation import *
from matplotlib.patches import Circle
from IPython.display import HTML

In [3]:
def plotObjective():
    plt.figure()

    delta = 0.025
    xx = np.arange(-1.7, 1.7, delta)
    yy = np.arange(-1.6, 1.6, delta)
    X, Y = np.meshgrid(xx, yy)
    Z = (1-X)**2 + (Y-X**2)**2
    plt.contour(X, Y, Z, levels=100)

    plt.gca().set_xlim([-1.7, 1.7])
    plt.gca().set_ylim([-1.5, 1.5])
    plt.gca().set_aspect('equal')

def createAnimation(opti_f):
    xx_sol = []
    yy_sol = []
    slopes = np.linspace(-2.0, 2.0, 30)
    for slope in slopes:
        x, y = opti_f(slope)
        xx_sol.append(x)
        yy_sol.append(y)
    
    plotObjective()
    plt.plot(np.cos(np.linspace(0, 2*np.pi, 100)), 
             np.sin(np.linspace(0, 2*np.pi, 100)), 'r', linewidth=4)
    constraint, = plt.plot(np.linspace(-2.0, 2.0, 2), slopes[0]*np.linspace(-2.0, 2.0, 2), "r", linewidth=4)
    solution, = plt.plot(xx_sol[0], yy_sol[0], marker='o', color='k', markersize=10)
    
    def update_frame(i):
        constraint.set_data(np.linspace(-2.0, 2.0, 2), slopes[i]*np.linspace(-2.0, 2.0, 2))
        solution.set_data([float(xx_sol[i])], [float(yy_sol[i])])
        return constraint, solution,
    
    animation1 = FuncAnimation(plt.gcf(), update_frame, frames=range(len(slopes)))
    
    HTML(animation1.to_jshtml())
    return animation1

## Basic CasADi introduction: variables and functions

Lets define our first CasADi function:

$f\left( \begin{bmatrix} x_1 & x_2 & x_3 & x_4 & x_5 \end{bmatrix}^\top \right) = \begin{bmatrix} x_1 + x_2 + x_3\\ x_3^2 + x_4^2 + x_5^2\end{bmatrix}$

In [4]:
# Define x and y
x = MX.sym("x", 5, 1)
y = MX.sym("y", 2, 1)

# Assign the values
y[0] = x[0] + x[1] + x[2]
y[1] = sumsqr(x[2:])

# Define a function and evaluate it
f = Function("f", [x], [y])
f([1, 2, 3, 4, 5])

RuntimeError: Error in Function::Function for 'f' [MXFunction] at .../casadi/core/function.cpp:280:
.../casadi/core/function_internal.cpp:146: Error calling MXFunction::init for 'f':
.../casadi/core/mx_function.cpp:406: f::init: Initialization failed since variables [y] are free. These symbols occur in the output expressions but you forgot to declare these as inputs. Set option 'allow_free' to allow free variables.

Lets introduce a paremeter in our CasADi function:

$f\left( \begin{bmatrix} x_1 & x_2 & x_3 & x_4 & x_5 \end{bmatrix}^\top, \textcolor{blue}{p} \right) = \begin{bmatrix} x_1 + x_2 + x_3\\ x_3^2 + x_4^2 + x_5^2 + \textcolor{blue}{p}\end{bmatrix}$

In [5]:
# Define x and y
x = MX.sym("x", 5, 1)
y = MX(2, 1)

# Define p
p = 2

# Assign values
y[0] = x[0] + x[1] + x[2]
y[1] = sumsqr(x[2:]) + p

# Define the function and print it
f = Function("f", [x], [y])
print(f([1, 2, 3, 4, 5]))

# Repeat with a different value of p
p = 20
f = Function("f", [x], [y])
print(f([1, 2, 3, 4, 5]))

[6, 52]
[6, 52]


## Using CasADi's Algorithmic differentation

Computing a derivative using jacobian:

In [6]:
J = Function("J", [x], [jacobian(y, x)])
print(J)
print(J([1,2,3,4,5]))

J:(i0[5])->(o0[2x5,6nz]) MXFunction

[[1, 1, 1, 00, 00], 
 [00, 00, 6, 8, 10]]


Computing a second order derivative using hessian:

In [7]:
H = Function("H", [x], [hessian(transpose(y) @ y, x)[0]])
print(H)
print(H([1,2,3,4,5]))

H:(i0[5])->(o0[5x5,17nz]) MXFunction

[[2, 2, 2, 00, 00], 
 [2, 2, 2, 00, 00], 
 [2, 2, 282, 96, 120], 
 [00, 00, 96, 336, 160], 
 [00, 00, 120, 160, 408]]


## Storing, loading and code generating casadi functions

In [9]:
# loading a casadi function (you will create this one later)
quadcopter_dynamics = Function.load("quadcopter_dynamics.casadi")
print(quadcopter_dynamics)

# saving a casadi function
quadcopter_dynamics.save("quadcopter_dynamics.casadi")

# code generation
quadcopter_dynamics.generate("quadcopter_dynamics.c", {"with_header": True})

# compiling code-generated casadi functions
# gcc -fPIC -shared myFunction.c -o myFunction.so

quadcopter_dynamics:(x[9],u[4])->(xdot[9]) MXFunction


'quadcopter_dynamics.c'

## Opti example
(see https://web.casadi.org/blog/opti/)
Let's now define an optimization problem and solve it using CasADi Opti.

### Basic Rosenbrock example
\begin{align}
\min_{x,u} \quad (1-x)^2 + (y-x^2)^2
\end{align}

In [ ]:
# create an opti instance
opti = Opti()

# define optimization variables
x = opti.variable(1, 1)
y = opti.variable(1, 1)

# define the objective function
opti.minimize((1-x)**2 + (y-x**2)**2)

# set the solver
opti.solver("ipopt")
sol = opti.solve()

# show the output
print(f"Optimal solution: ({sol.value(x)}, {sol.value(y)})")

plotObjective()
plt.plot(sol.value(x), sol.value(y), 'k', marker='o', markersize=10)

### Let's add a simple constraint
\begin{align}
\min_{x,u} (1-x)^2 + (y-x^2)^2\\
\text{s.t.} \quad x^2 + y^2 = 1\quad 
\end{align}

In [ ]:
# create an opti instance
opti = Opti()

# define optimization variables
x = opti.variable(1, 1)
y = opti.variable(1, 1)

# define the objective function
opti.minimize((1-x)**2 + (y-x**2)**2)

# define the constraint
opti.subject_to(x**2 + y**2 == 1)

# set the solver
opti.solver("ipopt", {"print_time":False}, {"print_level":0})
sol = opti.solve()

# show the solution
print(f"Optimal solution: ({sol.value(x)}, {sol.value(y)})")
print(f"Lagrange multiplier value: {sol.value(opti.debug.lam_g)}")

plotObjective()
plt.plot(np.cos(np.linspace(0, 2*np.pi, 100)), 
         np.sin(np.linspace(0, 2*np.pi, 100)), 'r', linewidth=4.0)
plt.plot(sol.value(x), sol.value(y), 'k', marker='o', markersize=10)

### Let's add an inequality constraint also
\begin{align}
\min_{x,u} (1-x)^2 + (y-x^2)^2\\
\text{s.t.} \quad x^2 + y^2 = 1\\
y >= px
\end{align}

In [ ]:
# create an opti instance
opti = Opti()

# define optimization variables
x = opti.variable(1, 1)
y = opti.variable(1, 1)

# define a parameter
p = opti.parameter(1, 1)

# define the objective function
opti.minimize((1-x)**2 + (y-x**2)**2)

# define the constraints
opti.subject_to(x**2 + y**2 == 1)
opti.subject_to(y >= p*x)

# set the solver
opti.solver("ipopt", {"print_time":False}, {"print_level":0, "max_iter":3000})

# set the parameter value
opti.set_value(p, 2)

# solve the problem
sol = opti.solve()

# show the solution
print(f"Optimal solution: ({sol.value(x)}, {sol.value(y)})")

plotObjective()
plt.plot(np.cos(np.linspace(0, 2*np.pi, 100)), 
         np.sin(np.linspace(0, 2*np.pi, 100)), 'r', linewidth=4.0)
plt.plot(np.linspace(-2.0, 2.0, 2), 2.0*np.linspace(-2.0, 2.0, 2), "r", linewidth=4)
plt.plot(sol.value(x), sol.value(y), 'k', marker='o', markersize=10)

### More advanced options in Opti
Callbacks

In [ ]:
# create an opti instance
opti = Opti()

# define optimization variables
x = opti.variable(1, 1)
y = opti.variable(1, 1)

# define a parameter
p = opti.parameter(1, 1)

# define the objective function
opti.minimize((1-x)**2 + (y-x**2)**2)

# define the constraints
opti.subject_to(x**2 + y**2 == 1)
opti.subject_to(y >= p*x)

# set the solver
opti.solver("ipopt", {"print_time":False}, {"print_level":0, "max_iter":3000})

# define a callback
# opti.callback(lambda i : print(opti.debug.value(x), opti.debug.value(y)))

# def plotAll():
#     plotObjective()
#     plt.plot(np.cos(np.linspace(0, 2*np.pi, 100)), np.sin(np.linspace(0, 2*np.pi, 100)), 'r', linewidth=4.0)
#     plt.plot(np.linspace(-2.0, 2.0, 2), 2.0*np.linspace(-2.0, 2.0, 2), "r", linewidth=4)
#     plt.plot(opti.debug.value(x), opti.debug.value(y), 'k', marker='o', markersize=10)
# opti.callback(lambda i : plotAll())

# set the parameter value
opti.set_value(p, 2)

# solve the problem
opti.solve()

### More advanced options in Opti
Create an opti.to_function object to play around with the parameter p.

In [ ]:
# create an opti instance
opti = Opti()

# define optimization variables
x = opti.variable(1, 1)
y = opti.variable(1, 1)

# define a parameter
p = opti.parameter(1, 1)

# define the objective function
opti.minimize((1-x)**2 + (y-x**2)**2)

# define the constraints
opti.subject_to(x**2 + y**2 == 1)
opti.subject_to(y >= p*x)

# set the solver
opti.solver("ipopt", {"print_time":False}, {"print_level":0, "max_iter":3000})

# create function object
opti_f = opti.to_function("opti_f", [p], [x, y])

# solve the problem
sol_x, sol_y = opti_f(2.0)

# show the solution
print(f"Optimal solution: ({sol_x}, {sol_y})")

plotObjective()
plt.plot(np.cos(np.linspace(0, 2*np.pi, 100)), 
         np.sin(np.linspace(0, 2*np.pi, 100)), 'r', linewidth=4.0)
plt.plot(np.linspace(-2.0, 2.0, 2), 2.0*np.linspace(-2.0, 2.0, 2), "r", linewidth=4)
plt.plot(sol_x, sol_y, 'k', marker='o', markersize=10)

In [ ]:
animation1 = createAnimation(opti_f)
HTML(animation1.to_jshtml())

Provide initial guess to prevent convergence to local minimum.

In [ ]:
# create an opti instance
opti = Opti()

# define optimization variables
x = opti.variable(1, 1)
y = opti.variable(1, 1)

# define a parameter
p = opti.parameter(1, 1)

# define the objective function
opti.minimize((1-x)**2 + (y-x**2)**2)

# define the constraints
opti.subject_to(x**2 + y**2 == 1)
opti.subject_to(y >= p*x)

# set an initial guess
opti.set_initial(x, 0.5)
opti.set_initial(y, 2.0)

# set the solver
opti.solver("ipopt", {"print_time":False}, {"print_level":0, "max_iter":3000})

# create function object
opti_f = opti.to_function("opti_f", [p], [x, y])

animation2 = createAnimation(opti_f)
HTML(animation2.to_jshtml())

# Example problem: hanging chain

Now, lets consider a more elaborate example optimization problem. We want to find the shape of a chain, fixed on both ends. This can be achieved by representing the chain by masses attached to each other with simple springs. Every mass has a coordinate $(x_i, y_i)$ and a mass $m_i$.

To find the shape of the chain, we minimize the sum of the potential energy in all springs and the gravitational potential energy of all masses. This results in the cost function

$J = \frac{1}{2}\sum_{i=1}^{N-1}{D_i \left(\sqrt{(x_i - x_{i+1})^2 + (y_i - y_{i+1})^2} -\frac{L}{N}\right)^2} + g\sum_{i=1}^{N}{m_i y_i}$

where $D_i = 70N [N/m]$ is a spring constant, $L = 2.0m$ is the length of the chain at rest (if all springs are at their rest length), $g = 9.81 [m/s^2]$ is the gravitational constant. You can use $m_i = 0.1 [kg]$, but feel free to play around with this value (or use different masses for different parts of the chain).

One side of the chain is attached to the point $(-1, 1)$ and the other side is attached to the point $(1, 1)$.

There are also some obstacles present through which the chain cannot go

a) Solve the optimization problem to find the shape of the chain

b) Can you make the chain rest on top of the highest obstacle?

In [ ]:
class obstacle:
    def __init__(self, center_x, center_y, radius):
        self.center_x = center_x
        self.center_y = center_y
        self.radius = radius

obstacles = [obstacle(-0.7, 0.7, 0.1), 
             obstacle(0.1, 0.8, 0.1), 
             obstacle(0.8, 0.8, 0.1),
             obstacle(-0.5, 1.2, 0.1),
             obstacle(0.0, 1.5, 0.1)]

# Define problem parameters
N = 60
L = 2.0
p_left = [-1, 1]
p_right = [1, 1]
D = 70*N
m = 0.1

# Create opti instance
opti = Opti()

x = opti.variable(N, 1)
y = opti.variable(N, 1)
L = opti.parameter()
opti.set_value(L, 2)

obj = 0.0
for i in range(N):
    # gravitational potential energy
    obj += m*9.81*y[i]

    # potential energy in the springs
    if i < N-1:
        obj += 0.5*D*(sqrt((x[i]-x[i+1])**2 + (y[i]-y[i+1])**2) - L/N)**2

    # obstacle avoidance constraints
    for obs in obstacles:
        opti.subject_to((x[i] - obs.center_x)**2 + (y[i] - obs.center_y)**2 >= (obs.radius)**2)
    
opti.minimize(obj)

# end-point constraints
opti.subject_to(x[0] == p_left[0])
opti.subject_to(y[0] == p_left[1])
opti.subject_to(x[-1] == p_right[0])
opti.subject_to(y[-1] == p_right[1])

# initial guess
opti.set_initial(x, linspace(p_left[0], p_right[0], N))
opti.set_initial(y, 3)

opti.solver('ipopt')
sol = opti.solve()
opti_chain = opti.to_function("opti_chain", [L, y], [x, y])

plt.figure()
x_chain, y_chain = opti_chain(2.0, 1.0)
for obs in obstacles:
    plt.gca().add_patch(Circle((obs.center_x, obs.center_y), obs.radius, color='r', alpha=0.5))
plt.plot(x_chain, y_chain, 'b-o')
plt.plot(p_left[0], p_left[1], 'ko')
plt.plot(p_right[0], p_right[1], 'ko')
plt.axis('equal')
plt.show()